In [1]:
import pandas as pd
from pandas.io import gbq
import json
import numpy as np

import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)
from oauth2client.service_account import ServiceAccountCredentials
from apiclient.http import MediaFileUpload
from apiclient.discovery import build

import glob

# Engagement model

In [2]:
# first create a hitid custom dimension via Google Tag Manager - hitscope
# create a custom metric in Google Analytics - hit scope
# import data from Google Big Query
# score users who visit page y after page x
# create a Google Analytics import file
# import engagement scores to Google Analytics via Google Analytics Management API
# more info on: www.......com

## Import data 

###### Change the following variables in the query:
- project id
- table
- start and end of the timestamp
- page regex in the where statement

In [3]:
# import data GBQ, sessions on page x (STEP 1 in funnel)
conf = {
   'project_id': 'YOUR PROJECT ID',
   'table': 'YOUR GOOGLE BIGQUERY TABLE'
}
q = """
 SELECT
    CONCAT(fullVisitorId,STRING(visitId)) AS sessionId,
    hits.page.pagePath AS page,
    hits.hitNumber AS hitnumber
  FROM (TABLE_DATE_RANGE([123780675.ga_sessions_], TIMESTAMP('2018-11-02'), TIMESTAMP('2018-11-04')))
  WHERE
    REGEXP_MATCH(hits.page.pagePath,'FILL IN FIRST STEP IN FUNNEL')
  """
df_pageX = gbq.read_gbq(q, project_id=conf['project_id'], dialect='legacy')

###### Change the following variables in the query:
- project id
- table
- start and end of the timestamp
- page regex in the where statement

In [4]:
# import data GBQ, sessions on page y (STEP 2 in funnel)
conf = {
   'project_id': 'YOUR PROJECT ID',
   'table': 'YOUR GOOGLE BIGQUERY TABLE'
}
q = """
 SELECT
    CONCAT(fullVisitorId,STRING(visitId)) AS sessionId,
    hits.page.pagePath AS page,
    hits.hitNumber AS hitnumber,
  FROM (TABLE_DATE_RANGE([123780675.ga_sessions_], TIMESTAMP('2018-11-02'), TIMESTAMP('2018-11-04')))
  WHERE
    REGEXP_MATCH(hits.page.pagePath,'FILL IN SECOND STEP IN FUNNEL')
  """
df_pageY = gbq.read_gbq(q, project_id=conf['project_id'], dialect='legacy')

## Data processing

In [5]:
# merge two dataframes df_facilities & df_map
df = pd.merge(df_pageX, df_pageY,  how='left', left_on='sessionId', right_on = 'sessionId')

In [ ]:
df.head()

In [26]:
# keep only the sessions with first page x followed by page y
df = df.drop(df[df.hitnumber_x > df.hitnumber_y].index)

In [27]:
# drop rows with duplicates, drop if session visit funnel multiple times
df = df.drop_duplicates(subset=['sessionId', 'page_x', 'page_y'], keep='first')

## Data modelling

In [28]:
# create score variables per step in funnel 
score1 = 1
score2 = 10

In [29]:
# create a new column and assign variable score1 to all sessions with page x
df['score1'] = score1

In [30]:
# assign score 2 to sessions with page y, if not give score value 0
df['score2'] = np.where(df['page_y'].notnull(), score2, 0)

In [31]:
df.dtypes

sessionId       object
page_x          object
hitnumber_x      int64
page_y          object
hitnumber_y    float64
score1           int64
score2           int32
dtype: object

In [ ]:
df.head()

In [34]:
def cust_count(grp):
             grp['count_row'] = grp['sessionId'].count()
             return grp

In [35]:
# we create a new column which check if their are sessions who have multiple hits on page y
df = df.groupby(['sessionId']).apply(cust_count)

In [ ]:
df.head()

In [37]:
# divide the sessions with score 2 by the count of rows per session
df['score2'] = df['score2'] / df['count_row']
df['score2'] = df['score2'].fillna(0)

In [38]:
# function to calculate the total score
def total_score(score1, score2):
    total = score1 + score2
    return total

In [39]:
# sum total sore
df['total_score'] = total_score(df['score1'], df['score2'])

In [ ]:
df.head()

In [41]:
# function to calculate all total scores per sessionId + create new column 'ga:metric5'
def cust_sum(grp):
             grp['ga:metric5'] = grp['total_score'].sum()
             return grp

In [42]:
# group by sessionId and sum the total score
df_import = df.groupby(['sessionId']).apply(cust_sum)

In [43]:
# you will need a int to import, check your settings of the custom metric
# Change index 5 into your own index number
df_import['ga:metric5'] = df_import['ga:metric5'].astype(int)

In [ ]:
df_import.head()

In [45]:
# drop duplicates, keep one unique sessionId
df_import = df_import.drop_duplicates(['sessionId'], keep='last')

## Add last hitID of each session to df_import 

In [46]:
# import data GBQ, all sessionId's with dimension hitId. Change index=63 in to your own index number.
conf = {
   'project_id': 'YOUR PROJECT ID',
   'table': 'YOUR GOOGLE BIGQUERY TABLE'
}
q = """
SELECT
  CONCAT(fullVisitorId,STRING(visitId)) AS sessionId,
  hits.hitNumber as hitnumber,
  hits.customDimensions.value AS cd63
FROM (TABLE_DATE_RANGE([123780675.ga_sessions_], TIMESTAMP('2018-11-02'), TIMESTAMP('2018-11-04')))
WHERE
  hits.customDimensions.index=63
  AND hits.type = 'PAGE'
  """
df_hitId = gbq.read_gbq(q, project_id=conf['project_id'], dialect='legacy')

In [47]:
# change column name to 'ga:dimension63'. Change index=63 into your own index number.
df_hitId = df_hitId.rename(columns={'cd63': 'ga:dimension63'})

In [ ]:
df_hitId.head()

In [49]:
# we need one hitId of each session, lets keep only the last hitId
df_hitId = df_hitId.drop_duplicates(subset=['sessionId'], keep='last')

In [50]:
# merge two dataframes to add a column with the last and correct hitID
df_import = pd.merge(df_import, df_hitId,  how='left', left_on='sessionId', right_on= 'sessionId')

In [ ]:
df_import.head()

In [52]:
# function to keep/drop columns
def keep_cols(DataFrame, keep_these):
    """Keep only the columns [keep_these] in a DataFrame, delete
    all other columns. 
    """
    drop_these = list(set(list(DataFrame)) - set(keep_these))
    return DataFrame.drop(drop_these, axis = 1)

In [53]:
# we don't need all these columns, so keep only the columns which are necessary for the import. 
# Change index 5 into your own index number.
df_import = df_import.pipe(keep_cols, ['ga:dimension63', 'ga:metric5'])

In [54]:
# if needed, change order of the columns to prepare for import
df_import = df_import[['ga:dimension63', 'ga:metric5']]

In [ ]:
# check what is in our csv
df_import

In [365]:
# change 63 into your own index number.
df_import = df_import.dropna(subset=['ga:dimension63'])

In [62]:
df_import.head()

ga:dimension63    object
ga:metric5         int32
dtype: object

## Create chunks

In [57]:
# choose number of chunks you need, this is depending on the number of rows. Limit per chunk = 1000
number_of_chunks = 6

In [58]:
rows_chunk = len(df_import) / number_of_chunks
print("number of rows in chunk: {} rows".format(rows_chunk))

number of rows in chunk: 1485.8333333333333 rows


In [59]:
# split the csv file in to the number of chunks you need
for i, df_import in enumerate(np.array_split(df_import,number_of_chunks)):
    with open(f"out{i}.csv","w") as fo:
            fo.write(df_import.to_csv(index=False))

## Import via API

In [60]:
# max rows per API request 1000

In [61]:
# Batches for import
csv_filename1 = 'out0.csv'
csv_filename2 = 'out1.csv'
csv_filename3 = 'out2.csv'
csv_filename4 = 'out3.csv'
csv_filename5 = 'out4.csv'
csv_filename6 = 'out5.csv'

### change the variables into yours

In [63]:
def get_service(api_name, api_version, scopes, key_file_location):
    """Get a service that communicates to a Google API.

    Args:
    api_name: The name of the api to connect to.
    api_version: The api version to connect to.
    scope: A list auth scopes to authorize for the application.
    key_file_location: The path to a valid service account JSON key file.

    Returns:
    A service that is connected to the specified API.
    """

    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        key_file_location, scopes=scopes)

    # Build the service object.
    service = build(api_name, api_version, credentials=credentials)

    return service

scopes = ['https://www.googleapis.com/auth/analytics.edit',
          'https://www.googleapis.com/auth/analytics']

# fill in your own variables.
key_file_location = 'credentials/dataimport.json' # example file path
account_id = '6XXXXXXX' # example, fill in Google Analytics account id
web_property_id = 'UA-6XXXXXXX-1' # fill in UA tracking id
custom_data_source_id = "DN-xz23DR6as3sMml1jRdw" # example, fill in import id
analytics = get_service('analytics', 'v3', scopes, key_file_location)

### import the different batches

In [64]:
# batch 1
try:
    media = MediaFileUpload(csv_filename1,
                          mimetype='application/octet-stream',
                          resumable=False)
    daily_upload = analytics.management().uploads().uploadData(
        accountId=account_id,
        webPropertyId=web_property_id,
        customDataSourceId=custom_data_source_id,
        media_body=media).execute()
    
    print("Upload successful!")

except TypeError as error:
    # Handle errors in constructing a query.
    print('There was an error in constructing your query : %s' % error)

Upload successful!


In [112]:
# batch 2
time.sleep(3)
try:
    media = MediaFileUpload(csv_filename2,
                          mimetype='application/octet-stream',
                          resumable=False)
    daily_upload = analytics.management().uploads().uploadData(
        accountId=account_id,
        webPropertyId=web_property_id,
        customDataSourceId=custom_data_source_id,
        media_body=media).execute()
    
    print("Upload successful!")

except TypeError as error:
    # Handle errors in constructing a query.
    print('There was an error in constructing your query : %s' % error)

Upload successful!


In [113]:
# batch 3
time.sleep(3)
try:
    media = MediaFileUpload(csv_filename3,
                          mimetype='application/octet-stream',
                          resumable=False)
    daily_upload = analytics.management().uploads().uploadData(
        accountId=account_id,
        webPropertyId=web_property_id,
        customDataSourceId=custom_data_source_id,
        media_body=media).execute()
    
    print("Upload successful!")

except TypeError as error:
    # Handle errors in constructing a query.
    print('There was an error in constructing your query : %s' % error)

Upload successful!


In [114]:
# batch 4
time.sleep(3)
try:
    media = MediaFileUpload(csv_filename4,
                          mimetype='application/octet-stream',
                          resumable=False)
    daily_upload = analytics.management().uploads().uploadData(
        accountId=account_id,
        webPropertyId=web_property_id,
        customDataSourceId=custom_data_source_id,
        media_body=media).execute()
    
    print("Upload successful!")

except TypeError as error:
    # Handle errors in constructing a query.
    print('There was an error in constructing your query : %s' % error)

Upload successful!


In [115]:
# batch 5
time.sleep(3)
try:
    media = MediaFileUpload(csv_filename5,
                          mimetype='application/octet-stream',
                          resumable=False)
    daily_upload = analytics.management().uploads().uploadData(
        accountId=account_id,
        webPropertyId=web_property_id,
        customDataSourceId=custom_data_source_id,
        media_body=media).execute()
    
    print("Upload successful!")

except TypeError as error:
    # Handle errors in constructing a query.
    print('There was an error in constructing your query : %s' % error)

Upload successful!


In [116]:
# batch 6
time.sleep(3)
try:
    media = MediaFileUpload(csv_filename6,
                          mimetype='application/octet-stream',
                          resumable=False)
    daily_upload = analytics.management().uploads().uploadData(
        accountId=account_id,
        webPropertyId=web_property_id,
        customDataSourceId=custom_data_source_id,
        media_body=media).execute()
    
    print("Upload successful!")

except TypeError as error:
    # Handle errors in constructing a query.
    print('There was an error in constructing your query : %s' % error)

Upload successful!
